In [5]:
from src.constants import SIM_COV_DIM, SIM_ACT_DIM, SIM_OUT_DIM
from src.models import (
    AdaptiveLinearModel,
    BehaviouralCloning,
    BehaviouralCloningLSTM,
    BehaviouralCloningDeep,
    RCAL,
    CIRL,
)  # noqa: F401
from src.data_loading import generate_linear_dataset, get_centre_data
import numpy as np

In [7]:
training_data = generate_linear_dataset(10000, 50, seed=41310)
validation_data = generate_linear_dataset(1000, 50, seed=41310).get_whole_batch()

In [3]:
true_omega = training_data.betas

In [ ]:
len(true_omega[0][0][0])

In [4]:
true_omega[0][0][0]

array([-1.12186692,  1.17518288])

In [46]:
omegas = np.zeros((10000,50,5,2))

for i in range(10000):
    for j in range(50):
        for k in range(5):
            omegas[i,j,k] = true_omega[i][j][k]


In [11]:
hyperparams = {
    "covariate_size": 5,
    "action_size": 2,
    "outcome_size": 1,
    "memory_hidden_size": 32,
    "memory_layers": 1,
    "memory_dropout": 0,
    "memory_size": 16,
    "outcome_hidden_size": 32,
    "outcome_layers": 1,
    "inf_hidden_size": 16,
    "inf_layers": 1,
    "inf_dropout": 0.5,
    "inf_fc_size": 32,
    "hidden_size": 64,
}

In [78]:
model = AdaptiveLinearModel
model = model(**hyperparams)
model.fit(
    training_data,
    batch_size=100,
    epochs=2,
    learning_rate=0.01,
    validation_set=validation_data,
)

Epoch 1 average loss: 0.60079 (16.21 seconds)
Epoch 1 ACC: 0.97728
Epoch 1 AUC: 0.9905
Epoch 1 APR: 0.98788
Epoch 1 NLL: 0.09099
Epoch 2 average loss: 0.12008 (15.26 seconds)
Epoch 2 ACC: 0.85592
Epoch 2 AUC: 0.92267
Epoch 2 APR: 0.8914
Epoch 2 NLL: 0.64122


In [82]:
info = model.inspection(training_data.get_whole_batch())
omega1 = info["omega_1_posterior"]
omega1 = omega1.detach().numpy()

omega0 = info["omega_0_posterior"]
omega0 = omega0.detach().numpy()

In [75]:
omega1.shape

(10000, 50, 5)

In [84]:
omega_hat = np.zeros((10000,50,5,2))
omega_hat[:,:,:,0] = omega0
omega_hat[:,:,:,1] = omega1

In [85]:
((omegas - omega_hat)**2).mean()

1.9666802847335194

In [65]:
model = BehaviouralCloning
model = model(**hyperparams)
model.fit(
    training_data,
    batch_size=100,
    epochs=30,
    learning_rate=0.01,
    validation_set=validation_data,
)

Epoch 1 average loss: 0.65467 (0.49 seconds)
Epoch 1 ACC: 0.64256
Epoch 1 AUC: 0.50237
Epoch 1 APR: 0.50075
Epoch 1 NLL: 0.65195
Epoch 2 average loss: 0.65044 (0.39 seconds)
Epoch 2 ACC: 0.64256
Epoch 2 AUC: 0.49955
Epoch 2 APR: 0.5002
Epoch 2 NLL: 0.65198
Epoch 3 average loss: 0.65045 (0.4 seconds)
Epoch 3 ACC: 0.64256
Epoch 3 AUC: 0.4985
Epoch 3 APR: 0.49886
Epoch 3 NLL: 0.65196
Epoch 4 average loss: 0.65042 (0.39 seconds)
Epoch 4 ACC: 0.64256
Epoch 4 AUC: 0.50113
Epoch 4 APR: 0.5014
Epoch 4 NLL: 0.6521
Epoch 5 average loss: 0.65045 (0.4 seconds)
Epoch 5 ACC: 0.64256
Epoch 5 AUC: 0.49964
Epoch 5 APR: 0.49947
Epoch 5 NLL: 0.65212
Epoch 6 average loss: 0.65045 (0.49 seconds)
Epoch 6 ACC: 0.64256
Epoch 6 AUC: 0.50151
Epoch 6 APR: 0.50075
Epoch 6 NLL: 0.65218
Epoch 7 average loss: 0.65045 (0.41 seconds)
Epoch 7 ACC: 0.64256
Epoch 7 AUC: 0.50379
Epoch 7 APR: 0.50214
Epoch 7 NLL: 0.65198
Epoch 8 average loss: 0.6505 (0.4 seconds)
Epoch 8 ACC: 0.64256
Epoch 8 AUC: 0.50298
Epoch 8 APR: 0.502

In [66]:
om = list()
for param in model.linear.parameters():
    om.append(param)

In [41]:
om[0].detach().numpy().T.shape

(5, 2)

In [67]:
omega_hat = np.zeros((10000,50,5,2))

for i in range(10000):
    for j in range(50):
            omega_hat[i,j] = om[0].detach().numpy().T

In [68]:
((omegas - omega_hat)**2).mean()

0.9550398227441694

In [69]:
om[0].detach().numpy().T

array([[ 0.13360591,  0.16040416],
       [-0.11670476, -0.08233473],
       [-0.39442605, -0.42886418],
       [ 0.02830347,  0.04954521],
       [ 0.28697681,  0.25019741]])

In [71]:
omega_hat[0,0]

array([[ 0.64942544,  0.17334391],
       [-0.76233122, -0.1311509 ],
       [-0.59868606, -0.72222158],
       [-0.36611284, -0.20768936],
       [ 0.01381616,  0.2618125 ]])